In [1]:
import uuid 
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SilverVendas") \
    .config("spark.extraListeners", "io.openlineage.spark.agent.OpenLineageSparkListener") \
    .config("spark.openlineage.columnLineage.datasetLineageEnabled", "true") \
    .config("spark.openlineage.transport.type", "http") \
    .config("spark.openlineage.transport.url", "http://api:5000") \
    .config("spark.openlineage.namespace", "spark_integration") \
    .config("spark.openlineage.parentJobName", "customer-job") \
    .config("spark.openlineage.parentRunId", uuid.uuid4().__str__()) \
    .getOrCreate()

25/06/19 18:44:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
delta_path_vendas = "s3a://bronze/vendas/"
delta_path_pessoas = "s3a://bronze/pessoas/"

# Ler os dados
df_vendas = spark.read.format("delta").load(delta_path_vendas)
df_pessoas = spark.read.format("delta").load(delta_path_pessoas)

25/06/19 18:44:47 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
                                                                                

In [3]:
df_vendas.show()

25/06/19 18:44:54 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
                                                                                

+--------+----------+-----+--------------------+-----------+
|order_id| food_item|price|          order_time|id_customer|
+--------+----------+-----+--------------------+-----------+
|      11|    Salada|35.48|2025-06-19 08:24:...|          3|
|      12|   Lasanha|52.04|2025-06-19 18:34:...|          5|
|      13|     Pizza|76.65|2025-06-19 07:35:...|          1|
|      14|     Sushi|36.17|2025-06-19 02:05:...|          3|
|      15|   Lasanha| 20.8|2025-06-19 17:25:...|          5|
|      16|Hamburguer|75.32|2025-06-19 09:45:...|          1|
|      17|     Sushi|36.52|2025-06-19 02:16:...|          3|
|      18|     Pizza|56.59|2025-06-18 20:53:...|          1|
|      19|     Pizza|32.64|2025-06-18 19:21:...|          2|
|      20|   Lasanha|40.63|2025-06-19 01:15:...|          4|
|       1|    Salada|53.23|2025-06-19 03:42:...|          1|
|       2|   Lasanha|53.52|2025-06-19 02:25:...|          5|
|       3|Hamburguer|50.24|2025-06-19 12:05:...|          1|
|       4|Hamburguer|63.

In [4]:
df_pessoas.show()

[Stage 20:>                                                         (0 + 1) / 1]

+-----------+-------------+
|id_customer|customer_name|
+-----------+-------------+
|          1|          Ana|
|          2|       Carlos|
|          3|         João|
|          4|        Lucas|
|          5|        Maria|
+-----------+-------------+



In [5]:
from pyspark.sql.functions import col, current_date

df_joined = df_vendas.join(
    df_pessoas,
    on="id_customer",
    how="left"
).withColumnRenamed("order_id", "nk_order") \
.withColumnRenamed("food_item", "item") \
.withColumnRenamed("price", "price") \
.withColumnRenamed("order_time", "order_time") \
.withColumn("etl_date", current_date())  # Adiciona a data do processamento

In [6]:
df_joined.show()

+-----------+--------+----------+-----+--------------------+-------------+----------+
|id_customer|nk_order|      item|price|          order_time|customer_name|  etl_date|
+-----------+--------+----------+-----+--------------------+-------------+----------+
|          3|      11|    Salada|35.48|2025-06-19 08:24:...|         João|2025-06-19|
|          5|      12|   Lasanha|52.04|2025-06-19 18:34:...|        Maria|2025-06-19|
|          1|      13|     Pizza|76.65|2025-06-19 07:35:...|          Ana|2025-06-19|
|          3|      14|     Sushi|36.17|2025-06-19 02:05:...|         João|2025-06-19|
|          5|      15|   Lasanha| 20.8|2025-06-19 17:25:...|        Maria|2025-06-19|
|          1|      16|Hamburguer|75.32|2025-06-19 09:45:...|          Ana|2025-06-19|
|          3|      17|     Sushi|36.52|2025-06-19 02:16:...|         João|2025-06-19|
|          1|      18|     Pizza|56.59|2025-06-18 20:53:...|          Ana|2025-06-19|
|          2|      19|     Pizza|32.64|2025-06-18 19:2

In [7]:
# Caminho para o bucket MinIO (formato S3)
delta_path = "s3a://silver/fat_vendas"

# Escrever como tabela Delta
df_joined.write.format("delta").mode("overwrite").save(delta_path)

25/06/19 18:45:12 ERROR ContextFactory: Query execution is null: can't emit event for executionId 16
25/06/19 18:45:12 ERROR ContextFactory: Query execution is null: can't emit event for executionId 16
25/06/19 18:45:12 ERROR ContextFactory: Query execution is null: can't emit event for executionId 18
25/06/19 18:45:12 ERROR ContextFactory: Query execution is null: can't emit event for executionId 18
25/06/19 18:45:12 ERROR ContextFactory: Query execution is null: can't emit event for executionId 19
25/06/19 18:45:12 ERROR ContextFactory: Query execution is null: can't emit event for executionId 19
25/06/19 18:45:12 ERROR ContextFactory: Query execution is null: can't emit event for executionId 20
25/06/19 18:45:12 ERROR ContextFactory: Query execution is null: can't emit event for executionId 20
25/06/19 18:45:12 ERROR ContextFactory: Query execution is null: can't emit event for executionId 21
25/06/19 18:45:12 ERROR ContextFactory: Query execution is null: can't emit event for execu

In [8]:
spark.stop()